<a href="https://colab.research.google.com/github/kluo9/HM-personalized-fashion-recommendation/blob/main/hm_ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of ranking stage is to give rankings of the items recalled based on features from item and customer. 

The training is achieved with input as features from item and customers, and out = 1/0 based on whether this item is purchased ot not.

This becomes a classification problem, and it is very imbalanced since the items with output label 1 is very small compared to 0. So I generate the negative samples (out label = 0) so that the negative sample size is about 30 * the positive (out label = 1) sample size.

I set the training data set to be train4, which are transactions made between 2020-08-15 to 2020-08-23. I will use the customers during this period to train the ranking model. To be specific, each transaction in train4 will be a sample: the user embedding is then generated from train1 to train3. 


In [27]:
import pandas as pd
import numpy as np
import pickle
import datetime
from scipy import stats
from sklearn.preprocessing import OneHotEncoder

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_GT_vali", "rb") as fp: 
     user_vali_GT = pickle.load(fp)

In [4]:
with open("/content/gdrive/MyDrive/hm_recommendation/user_recall_vali", "rb") as fp: 
     user_vali_recall = pickle.load(fp)

In [5]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [6]:
 ! mkdir ~/.kaggle
 ! cp kaggle.json ~/.kaggle/
 ! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f transactions_train.csv
transaction_df = pd.read_csv('/content/transactions_train.csv.zip')

 98% 574M/584M [00:07<00:00, 92.7MB/s]
100% 584M/584M [00:07<00:00, 78.4MB/s]


In [8]:
transaction_df["t_dat"] = pd.to_datetime(transaction_df["t_dat"])
train1 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,9,8)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,16))]
train2 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,9,1)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,8))]
train3 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,8,23)) & (transaction_df['t_dat'] < datetime.datetime(2020,9,1))]
train4 = transaction_df.loc[(transaction_df["t_dat"] >= datetime.datetime(2020,8,15)) & (transaction_df['t_dat'] < datetime.datetime(2020,8,23))]

val = transaction_df.loc[transaction_df["t_dat"] >= datetime.datetime(2020,9,16)]

In [9]:
del transaction_df

# Featue engineering

I will split the train data into train_ref and train4, and use train_ref to generate features for train4 customers, and use train4 to train ranking model

In [10]:
train_ref = pd.concat([train1, train2, train3], axis=0)

Possible features for ranking:

Features from customers:
1. Customer embedding (remove those customers with no purchasing history)
2. Customer age
3. Customer club_member_status
4. Customer fashion_news_frequency

Features from items
1. Item embedding
2. Item product_group
3. Item color
4. Item section_name

## Customer features

Customer embedding for the training set 

In [11]:
# extract customers in train4:
train4_customer = train4['customer_id']

In [12]:
# read item embedding
with open("/content/gdrive/MyDrive/hm_recommendation/item_embedding", "rb") as fp: 
     item_embedding = pickle.load(fp)

For each customer in train4_customer, calculate customer embedding based on their previous purchases (in train_ref).

In [13]:
train_ref['article_id'] = train_ref['article_id'].astype("string")
items_per_user = train_ref.groupby(['customer_id'])['article_id'].apply(list) 

In [14]:
def customer_embedding(purchase_history, customer, item_vectors):
  customer_embed = [0] * 10
  if customer in purchase_history.keys():
    items = purchase_history[customer]
    for item in items:
      customer_embed = np.add(customer_embed, item_vectors[item])
    customer_embed = np.divide(customer_embed, len(items))
  return customer_embed


In [15]:
embedding_all_customer = []
for user in train4_customer:
  embedding_all_customer.append(customer_embedding(items_per_user, user, item_embedding))

Customer age, club_member_status and fashion_news_frequency

In [16]:
! kaggle competitions download -c h-and-m-personalized-fashion-recommendations -f customers.csv
customers_df = pd.read_csv('/content/customers.csv.zip')
customers_df["age"].fillna(int(stats.mode(customers_df["age"])[0]), inplace = True)

 91% 89.0M/97.9M [00:00<00:00, 114MB/s]
100% 97.9M/97.9M [00:01<00:00, 98.3MB/s]


In [25]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [31]:
user_age = pd.DataFrame()
user_age['customer_id'] = train4_customer.values
user_age = pd.merge(customers_df[['customer_id','age','club_member_status','fashion_news_frequency']], user_age, how='right', left_on = 'customer_id', right_on = 'customer_id')
user_age

,customer_id,age,club_member_status,fashion_news_frequency
0,000a9291e5aa007a10faf2177671a8501dade80e667921...,50.0,ACTIVE,NONE
1,000c0995dc7f3fbec41cf1cb2c535f6e2ae99c0fe351e4...,49.0,PRE-CREATE,NONE
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,22.0,ACTIVE,Regularly
3,001ea4e9c54f7e9c88811260d954edc059d596147e1cf8...,25.0,ACTIVE,NONE
4,001ea4e9c54f7e9c88811260d954edc059d596147e1cf8...,25.0,ACTIVE,NONE
...,...,...,...,...
263581,ffed7546e705aa2c38b4cfff36dd4dd578aa141bdce668...,25.0,ACTIVE,NONE
263582,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE
263583,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE
263584,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE


Now need to turn club_member_status and fashion_news_frequency into one-hot encoding

In [32]:
one_hot = OneHotEncoder()
encoded_club_member_status = one_hot.fit_transform(user_age[['club_member_status']])
user_age[one_hot.categories_[0]] = encoded_club_member_status.toarray()

encoded_fashion_news_frequency = one_hot.fit_transform(user_age[['fashion_news_frequency']])
user_age[one_hot.categories_[0]] = encoded_fashion_news_frequency.toarray()

In [33]:
user_age

,customer_id,age,club_member_status,fashion_news_frequency,ACTIVE,LEFT CLUB,PRE-CREATE,NaN,Monthly,NONE,Regularly
0,000a9291e5aa007a10faf2177671a8501dade80e667921...,50.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,000c0995dc7f3fbec41cf1cb2c535f6e2ae99c0fe351e4...,49.0,PRE-CREATE,NONE,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,22.0,ACTIVE,Regularly,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,001ea4e9c54f7e9c88811260d954edc059d596147e1cf8...,25.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,001ea4e9c54f7e9c88811260d954edc059d596147e1cf8...,25.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
263581,ffed7546e705aa2c38b4cfff36dd4dd578aa141bdce668...,25.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
263582,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
263583,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
263584,fff799672ab2fa65916b4a362575550fbc3ee9a01d0508...,23.0,ACTIVE,NONE,1.0,0.0,0.0,0.0,0.0,1.0,0.0
